# SW Draconis : observing blocks


Philippe Hekkers, Siméon Deschaux-Beaume, Margaux Vandererven

In [1]:
import numpy as np
from astropy.time import Time
from astropy.coordinates import EarthLocation, get_sun, get_moon, AltAz
from pandas import DataFrame
import matplotlib.pyplot as plt
import astropy.units as u
from astropy.coordinates import SkyCoord
from astroplan import Observer

#### Informations du T5 :

In [2]:
longitude = -113.496040 *u.deg
latitude = 37.103080*u.deg
elevation = 1570 * u.m
location = EarthLocation(lat = latitude, lon = longitude, height = elevation)

T5 = Observer(name='T5', 
             location=location)

#### Informations sur SW Dra :

In [3]:
sw_dra = SkyCoord.from_name('SW Dra') #coordonnées de SW Dra dans Simbad
# print(sw_dra)

p = 0.56966993 #période en jours

periode = p*24*60 #période en minutes

#### Phases souhaitées : 

In [4]:
PHASE = np.array([0.000, 0.200, 0.410, 0.600, 0.700, 0.785, 0.840, 0.885, 0.910, 0.930]) #phase à laquelle on souhaite prendre une image
phase = PHASE * periode

print('période en minute : ', periode, 'min \n')
print("phases souhaitées de l'étoile : " , PHASE, '\n')
print('minutes de prise : ', phase, '\n')

période en minute :  820.3246992 min 

phases souhaitées de l'étoile :  [0.    0.2   0.41  0.6   0.7   0.785 0.84  0.885 0.91  0.93 ] 

minutes de prise :  [  0.         164.06493984 336.33312667 492.19481952 574.22728944
 643.95488887 689.07274733 725.98735879 746.49547627 762.90197026] 



#### Détermination des époques : 

Le début est déterminé par AAVSO pour le maximum de luminosité de SW Dra.

In [5]:
beginn = Time('2023-03-27T00:14:00', format='isot', scale='utc')
begin = np.float(beginn.jd)
endd = Time('2023-5-30T00:00:00', format='isot', scale='utc')
end = np.float(endd.jd)

C:\Users\marga\AppData\Local\Temp/ipykernel_9396/1610955171.py:2: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  begin = np.float(beginn.jd)
C:\Users\marga\AppData\Local\Temp/ipykernel_9396/1610955171.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  end = np.float(endd.jd)


In [6]:
# on découpe le temps pour chacune des phases. 

T_t = []
PHASE_tout = []
for i in range(10):
    phase_i = phase[i]/(60*24)
    t_i = np.arange(begin+phase_i, end, p).tolist()
    T_t.append(t_i)
    for j in range(len(t_i)):
        PHASE_tout.append(PHASE[i])
    
T = sum(T_t,[])

In [7]:
t = Time(np.array(T), format='jd')
timee = t.isot
time = Time(timee, format='isot', scale='utc')

UTC_offset = 6*u.hour
time_Utah = time - UTC_offset
time_Utah_jd = time_Utah.jd
print(time_Utah_jd)

[2460030.25972222 2460030.82939215 2460031.39906208 ... 2460092.8835376
 2460093.45320753 2460094.02287747]


#### Distance à la Lune de SW Dra : 

In [8]:
# position de la lune pour chaque époque

moon = get_moon(t, location = location)

data_dict = { 'Right ascension':moon.ra.T,
              'Declinaison': moon.dec.T,
          }
df = DataFrame(data_dict)
df.head()
print(df)

      Right ascension  Declinaison
0           70.122828    25.261492
1           78.252994    25.904971
2           85.231780    27.286151
3           94.250493    27.221132
4          100.509335    27.421138
...               ...          ...
1117       157.239373    13.518725
1118       164.030827    10.918057
1119       170.266999     7.653223
1120       175.966898     4.636910
1121       183.086476     1.321287

[1122 rows x 2 columns]


In [9]:
# on calcule la distance à la lune de SW Dra

ra = df['Right ascension']
dec = df['Declinaison']

angle_separation = np.zeros(len(t.isot))

for i in range(len(t.isot)):
    c2 = SkyCoord(ra[i]*u.deg, dec[i]*u.deg, frame='icrs')
    sep = sw_dra.separation(c2)
    angle_separation[i] = sep.radian
    

angle_separation2 = angle_separation * 180 / np.pi # On convertit en degrés

#### Détermination des phases de la Lune : 

In [10]:
def moon_phase_angle(time, ephemeris=None):
    """
    Calculate lunar orbital phase in radians.

    Parameters
    ----------
    time : `~astropy.time.Time`
        Time of observation

    ephemeris : str, optional
        Ephemeris to use.  If not given, use the one set with
        `~astropy.coordinates.solar_system_ephemeris` (which is
        set to 'builtin' by default).

    Returns
    -------
    i : `~astropy.units.Quantity`
        Phase angle of the moon [radians]
    """
    sun = get_sun(time)
    moon = get_moon(time, ephemeris=ephemeris)
    elongation = sun.separation(moon)
    return np.arctan2(sun.distance*np.sin(elongation),
                      moon.distance - sun.distance*np.cos(elongation))



def moon_illumination(time, ephemeris=None):
    """
    Calculate fraction of the moon illuminated.

    Parameters
    ----------
    time : `~astropy.time.Time`
        Time of observation

    ephemeris : str, optional
        Ephemeris to use.  If not given, use the one set with
        `~astropy.coordinates.solar_system_ephemeris` (which is
        set to 'builtin' by default).

    Returns
    -------
    k : float
        Fraction of moon illuminated
    """
    i = moon_phase_angle(time, ephemeris=ephemeris)
    k = (1 + np.cos(i))/2.0
    return k.value

In [11]:
moon_illumination1 = moon_illumination(t, ephemeris=None)

#### Détermination de la position du Soleil et de SW Dra :

In [12]:
altaz2 = AltAz(obstime=t,
                location=location) #depuis le T5

sun2 = get_sun(t) #position du Soleil
sun2_altaz = sun2.transform_to(altaz2) #position du Soleil depuis le T5
sw_dra2_altaz = sw_dra.transform_to(altaz2) #position de SW Dra depuis le T5

#### Résumé des données :  

In [13]:
data = np.array([T, time_Utah_jd, angle_separation2, moon_illumination1, sun2_altaz.alt, sw_dra2_altaz.alt, PHASE_tout])
date = data.T

In [14]:
TMM = Time(date[:,0], format = 'jd')

In [15]:
data_dict3 = { 'Date et heure (UTC)':TMM.isot,
               'Date et heure local en Utah (UTC-6)' : time_Utah.isot,
               'Distance à la Lune' : angle_separation2,
               'Phase de la Lune' : moon_illumination1,
               'Hauteur du Soleil' : sun2_altaz.alt,
               'Hauteur de SW Dra' : sw_dra2_altaz.alt
              }
df3 = DataFrame(data_dict3)
df3.head()
print(df3)

          Date et heure (UTC) Date et heure local en Utah (UTC-6)  \
0     2023-03-27T00:14:00.000             2023-03-26T18:14:00.000   
1     2023-03-27T13:54:19.482             2023-03-27T07:54:19.482   
2     2023-03-28T03:34:38.964             2023-03-27T21:34:38.964   
3     2023-03-28T17:14:58.446             2023-03-28T11:14:58.446   
4     2023-03-29T06:55:17.928             2023-03-29T00:55:17.928   
...                       ...                                 ...   
1117  2023-05-27T11:51:38.685             2023-05-27T05:51:38.685   
1118  2023-05-28T01:31:58.167             2023-05-27T19:31:58.167   
1119  2023-05-28T15:12:17.649             2023-05-28T09:12:17.649   
1120  2023-05-29T04:52:37.131             2023-05-28T22:52:37.131   
1121  2023-05-29T18:32:56.613             2023-05-29T12:32:56.613   

      Distance à la Lune  Phase de la Lune  Hauteur du Soleil  \
0              74.372878          0.301164          18.419652   
1              71.249743          0.35464

#### Application des contraintes de visibilité : 

In [22]:
date1 = date[~(date[:,2] <= 30)] #distance à la lune supérieure à 30°
date2 = date1[~(date1[:,3] > 0.9)] #une lune éclairée à 80% ou moins

date3 = date2[~(date2[:,4] >= -18)] #hauteur du soleil <-18° pour se situer dans le crépuscule astronomique


DATE = date3[~(date3[:,5] <= 30)] #hauteur dans le ciel de sw dra

In [23]:
TM = Time(DATE[:,0], format = 'jd')
TU = Time(DATE[:,1], format = 'jd')

In [24]:
data_dict4 = { 'Date et heure (UTC)':TM.isot,
               'Date et heure local en Utah (UTC-6)' : TU.iso,
               'Distance à la Lune (deg)' : DATE[:,2],
               'Phase de la Lune' : DATE[:,3],
               'Hauteur du Soleil (deg)' : DATE[:,4],
               'Hauteur de SW Dra (deg)' : DATE[:,5],
               "Phase de SW Dra" : DATE[:,6]
              }
df4 = DataFrame(data_dict4)
df4.head()
df5 = df4.sort_values(by=['Date et heure (UTC)'])
print(df5)

df5.to_excel('on pleure.xlsx', header=True, index=False)

         Date et heure (UTC) Date et heure local en Utah (UTC-6)  \
55   2023-03-27T05:50:19.988             2023-03-26 23:50:19.988   
83   2023-03-27T08:26:11.689             2023-03-27 02:26:11.689   
109  2023-03-27T09:48:13.637             2023-03-27 03:48:13.637   
135  2023-03-27T10:57:57.293             2023-03-27 04:57:57.293   
161  2023-03-27T11:43:04.365             2023-03-27 05:43:04.365   
..                       ...                                 ...   
82   2023-05-28T08:05:43.518             2023-05-28 02:05:43.518   
244  2023-05-29T04:36:12.741             2023-05-28 22:36:12.741   
273  2023-05-29T04:52:37.131             2023-05-28 22:52:37.131   
26   2023-05-29T05:50:02.495             2023-05-28 23:50:02.495   
54   2023-05-29T08:34:06.391             2023-05-29 02:34:06.391   

     Distance à la Lune (deg)  Phase de la Lune  Hauteur du Soleil (deg)  \
55                  73.380597          0.322922               -42.983328   
83                  72.848298  